In [20]:
import openai
import json
import requests
from typing import List, Dict, Any

In [21]:
import openai
import json
import requests
from typing import List, Dict, Any

client = openai.OpenAI()
messages = []
BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"

# --- 1. Tool Functions (Logic) ---


def get_popular_movies() -> str:
    """Fetch a list of currently popular movies."""
    try:
        response = requests.get(f"{BASE_URL}/movies", timeout=10)
        response.raise_for_status()
        # We return as a string because OpenAI tools expect string content
        return json.dumps(response.json()[:10])  # Limiting to 10 for token efficiency
    except Exception as e:
        return json.dumps({"error": str(e)})


def get_movie_details(movie_id: int) -> str:
    """Fetch detailed information about a specific movie by its ID."""
    try:
        response = requests.get(f"{BASE_URL}/movies/{movie_id}", timeout=10)
        response.raise_for_status()
        return json.dumps(response.json())
    except Exception as e:
        return json.dumps({"error": f"Movie ID {movie_id} not found."})


def get_movie_credits(movie_id: int) -> str:
    """Fetch the cast and crew for a specific movie ID."""
    try:
        response = requests.get(f"{BASE_URL}/movies/{movie_id}/credits", timeout=10)
        response.raise_for_status()
        return json.dumps(response.json())
    except Exception as e:
        return json.dumps({"error": f"Credits for movie {movie_id} not found."})

In [22]:
# --- 2. Configuration ---

FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_movie_credits": get_movie_credits,
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "Get a list of popular movies currently in theaters.",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "Get detailed info (overview, runtime, etc.) for a movie. Requires a movie_id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "The ID of the movie.",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_credits",
            "description": "Get the cast and crew of a movie. Requires a movie_id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "The ID of the movie.",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
]

In [23]:
# --- 3. Agent Logic ---


def process_ai_response(message):
    if message.tool_calls:
        # 1. Record the assistant's request to use tools
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": "function",
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments,
                        },
                    }
                    for tc in message.tool_calls
                ],
            }
        )

        # 2. Execute each tool call
        for tool_call in message.tool_calls:
            fname = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"[*] Calling Tool: {fname}({args})")

            func = FUNCTION_MAP.get(fname)
            result = func(**args)

            # 3. Append tool result back to history
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": fname,
                    "content": result,
                }
            )

        # 4. Recursively call AI to analyze results
        return call_ai()
    else:
        # Final response
        content = message.content
        messages.append({"role": "assistant", "content": content})
        print(f"AI: {content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [24]:
# --- 4. Loop ---

print("--- Movie Agent Active (type 'quit' to exit) ---")
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break

    messages.append({"role": "user", "content": user_input})
    call_ai()

--- Movie Agent Active (type 'quit' to exit) ---
[*] Calling Tool: get_popular_movies({})
AI: 현재 인기 있는 영화 목록은 다음과 같습니다:

1. **Mercy**
   - 개봉일: 2026-01-20
   - 개요: 가까운 미래, 한 형사가 아내를 살해한 혐의로 재판을 받고 있습니다. 그는 자신의 결백을 증명하기 위해 한 고급 AI 판사에게 90분의 시간을 주어야 합니다.
   - 평점: 7.054
   - ![포스터](https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg)

2. **28 Years Later: The Bone Temple**
   - 개봉일: 2026-01-14
   - 개요: Dr. Kelson은 세상을 변화시킬 수 있는 놀라운 관계에 휘말리게 되고, Spike는 Jimmy Crystal과의 만남에서 벗어날 수 없는 악몽을 경험합니다.
   - 평점: 7.2
   - ![포스터](https://image.tmdb.org/t/p/w780/kK1BGkG3KAvWB0WMV1DfOx9yTMZ.jpg)

3. **The Orphans**
   - 개봉일: 2025-08-20
   - 개요: 어린 시절 친구였던 Gab과 Driss가 재회하지만, 한 사건으로 인해 서로를 돕기 위해 팀을 이루어야 합니다.
   - 평점: 6.043
   - ![포스터](https://image.tmdb.org/t/p/w780/hP7mjZr2SVfjAorlRHTdV1XZmHY.jpg)

4. **A Woman Scorned**
   - 개봉일: 2025-06-09
   - 개요: Jas는 가족 여행 중 공격을 당한 여동생의 복수를 다짐합니다.
   - 평점: 6.0
   - ![포스터](https://image.tmdb.org/t/p/w780/dlOSBiNULMPzKIze84LDjvEN9z1.jpg)

5. **Deathstalker*